In [1]:
import pandas as pd
# cols_to_load = ['row_id', 'user_id', 'answered_correctly', 'content_id', 'prior_question_had_explanation', 'prior_question_elapsed_time']
cols_to_load = ['row_id', 'user_id', 'answered_correctly', 'content_id']
df = pd.read_csv("riiid-test-answer-prediction/train.csv")[cols_to_load]

In [2]:
import numpy as np
np.min(df[df.answered_correctly != -1]['content_id'].unique())
#0 -> 13522

0

In [103]:
train = df.sample(100000)
train.head()

NameError: name 'df' is not defined

In [66]:
del df

In [5]:
train = train.drop(columns=['row_id'])

In [8]:
train['answered_correctly'] = train['answered_correctly'].replace(1,2)
train['answered_correctly'] = train['answered_correctly'].replace(0,1)

In [82]:
train = train[train.answered_correctly != -1].groupby('user_id').apply(lambda x: (x['content_id'],x['answered_correctly']))

In [156]:
train = train[0:500]

In [157]:
train_full = []
for item in train:
    train_list = [0] * 13523
    for i in range(0,len(item),2):
        train_list[item[i].values[0]] = item[i+1].values[0]
    train_full.append(train_list)

In [158]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset

In [159]:
class network2(nn.Module):
    
    def __init__(self , num_item):
        
        super().__init__()
        self.fc1 = nn.Linear(num_item , 1024)
        self.fc2 = nn.Linear(1024 , 512)
        self.fc3 = nn.Linear(512 , 256)
        self.fc4 = nn.Linear(256 , 512)
        self.fc5 = nn.Linear(512 , 1024)
        self.out = nn.Linear(1024 , num_item)
        
    def forward(self , x):
        
        x = F.selu(self.fc1(x))
        x = F.selu(self.fc2(x))
        x = F.selu(self.fc3(x))
        x = F.selu(self.fc4(x))
        x = F.selu(self.fc5(x))
        x = self.out(x)
        return x

In [160]:
model = network2(13523).float()
criterion = nn.MSELoss()
optimizer = torch.optim.AdamW(model.parameters(),lr=0.001)

In [163]:
num_epochs = 3
for epoch in range(num_epochs):
    total_loss = 0 
    batch_data = TensorDataset(torch.tensor(train_full))
    dataloader = DataLoader(batch_data, batch_size=128, shuffle=True)
    for data in dataloader:
        data = data[0].float()
        # ===================forward====================
        output = model(data)
        mask = data != 0
        loss = criterion(output * mask.float(), data)
        total_loss += loss
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print('epoch [{}/{}], loss:{:.4f}' .format(epoch + 1, num_epochs, total_loss))

epoch [1/3], loss:0.0001
epoch [2/3], loss:0.0000
epoch [3/3], loss:0.0000


In [4]:
df['prior_question_had_explanation'] = df['prior_question_had_explanation'].astype('bool')

In [5]:
train = df.sample(1000000)

In [6]:
train.head()

,row_id,user_id,answered_correctly,content_id,prior_question_had_explanation,prior_question_elapsed_time
87341296,87341296,1858251086,1,10688,True,16000.0
11529006,11529006,250025288,1,2851,True,25333.0
30690375,30690375,658827158,0,7341,True,29250.0
82308154,82308154,1752472942,0,4118,True,4000.0
63365164,63365164,1347409700,1,960,True,16000.0


In [7]:
import numpy as np
user_df = train[train.answered_correctly != -1].groupby('user_id').agg({'answered_correctly': ['count', 'mean']}).reset_index()
user_df.columns = ['user_id', 'user_questions_amount', 'user_correct_rate']

user_lect = train.groupby(["user_id", "answered_correctly"]).size().unstack()
user_lect.columns = ['Lecture', 'Wrong', 'Right']
user_lect['Lecture'] = user_lect['Lecture'].fillna(0)

user_lect = user_lect.astype('Int64')
user_lect['watches_lecture'] = np.where(user_lect.Lecture > 0, 1, 0)
user_lect = user_lect.reset_index()
user_lect = user_lect[['user_id', 'watches_lecture']]

user_df = user_df.merge(user_lect, on = "user_id", how = "left")
del user_lect
user_df.head()

,user_id,user_questions_amount,user_correct_rate,watches_lecture
0,115,3,0.666667,0
1,5382,3,0.666667,0
2,8623,1,1.000000,0
3,12741,3,0.333333,0
4,13134,12,0.666667,0


In [8]:
content_df = train[train.answered_correctly != -1].groupby('content_id').agg({'answered_correctly': ['count', 'mean']}).reset_index()
content_df.columns = ['content_id', 'questions_answered_amount', 'question_correct_rate']
content_df.head()

,content_id,questions_answered_amount,question_correct_rate
0,0,70,0.942857
1,1,46,0.934783
2,2,469,0.545842
3,3,227,0.784141
4,4,316,0.591772


In [9]:
train = train[train.answered_correctly != -1]

mean_prior = train.prior_question_elapsed_time.astype("float64").mean()

In [10]:
from sklearn.preprocessing import LabelEncoder
label_enc = LabelEncoder()

train = train.merge(user_df, on = "user_id", how = "left")
train = train.merge(content_df, on = "content_id", how = "left")
train['questions_answered_amount'].fillna(0, inplace = True)
train['question_correct_rate'].fillna(0.5, inplace = True)
train['watches_lecture'].fillna(0, inplace = True)
train['user_questions_amount'].fillna(0, inplace = True)
train['user_correct_rate'].fillna(0.5, inplace = True)
train['prior_question_elapsed_time'].fillna(mean_prior, inplace = True)
train['prior_question_had_explanation'].fillna(False, inplace = True)
train['prior_question_had_explanation'] = label_enc.fit_transform(train['prior_question_had_explanation'])
train[['questions_answered_amount', 'user_questions_amount']] = train[['questions_answered_amount', 'user_questions_amount']].astype(int)
train.sample(5)

,row_id,user_id,answered_correctly,content_id,prior_question_had_explanation,prior_question_elapsed_time,user_questions_amount,user_correct_rate,watches_lecture,questions_answered_amount,question_correct_rate
769507,76520276,1628975888,0,10169,1,49000.0,64,0.609375,1,41,0.658537
459661,90933424,1930286578,1,3395,1,25667.0,52,0.942308,1,71,0.718310
524518,43376119,918781738,1,7271,0,55200.0,54,0.703704,1,84,0.500000
132985,46207013,978472791,0,2417,1,12000.0,91,0.637363,1,65,0.630769
291144,21991719,472397262,0,3169,1,26333.0,23,0.782609,0,74,0.391892


In [11]:
features = ['user_questions_amount', 'user_correct_rate', 'questions_answered_amount', 'question_correct_rate', 'prior_question_elapsed_time', 'prior_question_had_explanation']

y_train = train[0:800000]['answered_correctly']
X_train = train[0:800000][features]

y_test = train[800000:1000000]['answered_correctly']
X_test = train[800000:1000000][features]

In [15]:
import xgboost
params = {}
params['objective'] = 'binary:logistic'
params['eval_metric'] = 'logloss'
params['eta'] = 0.04
params['max_depth'] = 8
params['learning_rate'] = 0.01

d_train = xgboost.DMatrix(X_train, label=y_train)
d_valid = xgboost.DMatrix(X_test, label=y_test)

watchlist = [(d_train, 'train'), (d_valid, 'valid')]

bst = xgboost.train(params, d_train, 30, watchlist, verbose_eval=10)
y_pred = bst.predict(xgboost.DMatrix(X_test))

[0]	train-logloss:0.68888	valid-logloss:0.68905
[10]	train-logloss:0.65171	valid-logloss:0.65168
[20]	train-logloss:0.62065	valid-logloss:0.62082
[29]	train-logloss:0.59735	valid-logloss:0.59732


In [16]:
from sklearn.metrics import accuracy_score, f1_score
y_pred_decision = [0] * len(y_pred)
for i in range(len(y_pred)):
    if(y_pred[i] <= 0.5):
        y_pred_decision[i] = 0
    else:
        y_pred_decision[i] = 1
print('Accuracy: %f' % accuracy_score(y_test, y_pred_decision))
print('f1-score: %f' % f1_score(y_test, y_pred_decision))

Accuracy: 0.781831
f1-score: 0.843382
